In [1]:
!pip install --upgrade numpy
!pip install --upgrade pandas
!pip install --upgrade tensorflow
!pip install --upgrade h5py

  Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl (14.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.24.4 which is incompatible.


  Using cached tensorflow-2.13.1-cp38-cp38-win_amd64.whl.metadata (2.6 kB)
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
  Using cached numpy-1.24.3-cp38-cp38-win_amd64.whl.metadata (5.6 kB)
Using cached numpy-1.24.3-cp38-cp38-win_amd64.whl (14.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fairlearn 0.11.0 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.


In [3]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

In [4]:
dataset_orig = GermanDataset(
    protected_attribute_names=['age'],                           
    privileged_classes=[lambda x: x >= 25],     
    features_to_drop=['personal_status', 'sex'] 
   )

In [5]:
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

In [6]:
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

In [7]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

Difference in mean outcomes between unprivileged and privileged groups = -0.129093


In [8]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)  

In [9]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)

print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

Difference in mean outcomes between unprivileged and privileged groups = 0.000000


In [10]:
from aif360.datasets import GermanDataset

# Load the dataset to inspect its columns
dataset = GermanDataset()
print(dataset.feature_names)

['month', 'credit_amount', 'investment_as_income_percentage', 'residence_since', 'age', 'number_of_credits', 'people_liable_for', 'sex', 'status=A11', 'status=A12', 'status=A13', 'status=A14', 'credit_history=A30', 'credit_history=A31', 'credit_history=A32', 'credit_history=A33', 'credit_history=A34', 'purpose=A40', 'purpose=A41', 'purpose=A410', 'purpose=A42', 'purpose=A43', 'purpose=A44', 'purpose=A45', 'purpose=A46', 'purpose=A48', 'purpose=A49', 'savings=A61', 'savings=A62', 'savings=A63', 'savings=A64', 'savings=A65', 'employment=A71', 'employment=A72', 'employment=A73', 'employment=A74', 'employment=A75', 'other_debtors=A101', 'other_debtors=A102', 'other_debtors=A103', 'property=A121', 'property=A122', 'property=A123', 'property=A124', 'installment_plans=A141', 'installment_plans=A142', 'installment_plans=A143', 'housing=A151', 'housing=A152', 'housing=A153', 'skill_level=A171', 'skill_level=A172', 'skill_level=A173', 'skill_level=A174', 'telephone=A191', 'telephone=A192', 'fore

In [11]:
dataset_orig = GermanDataset(
    protected_attribute_names=['credit_amount'],  
    privileged_classes=[lambda x: x > 4000],     
    features_to_drop=['personal_status', 'age', 'sex']  
)
print("Dataset loaded with 'credit_amount' as the protected attribute!")

Dataset loaded with 'credit_amount' as the protected attribute!


In [12]:
# Split the dataset
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

# Define privileged & unprivileged groups
privileged_groups = [{'credit_amount': 1}]
unprivileged_groups = [{'credit_amount': 0}]

print("Dataset successfully split!")

Dataset successfully split!


In [13]:
metric_orig_train = BinaryLabelDatasetMetric(
    dataset_orig_train,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)
print("Difference in mean outcomes between unprivileged & privileged groups before mitigation = %f" % metric_orig_train.mean_difference())


Difference in mean outcomes between unprivileged & privileged groups before mitigation = 0.149963


In [14]:
# Apply Reweighing
RW = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

print("Bias mitigation applied using Reweighing algorithm!")


Bias mitigation applied using Reweighing algorithm!


In [15]:
# Compute fairness metric after mitigation
metric_transf_train = BinaryLabelDatasetMetric(
    dataset_transf_train,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)
print("Difference in mean outcomes between unprivileged and privileged groups after mitigation = %f" % metric_transf_train.mean_difference())

Difference in mean outcomes between unprivileged and privileged groups after mitigation = 0.000000
